In [1]:
%load_ext autoreload
%autoreload 2
    
import sys
import copy

sys.path.append("../lib/")

# from etna.experimental.classification.feature_extraction import TSFreshFeatureExtractor
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier, Pool
from skimage.util.shape import view_as_windows
import scipy.linalg
from IPython.display import clear_output

tqdm.pandas()

import prepare_dataset
import feature_preparation

In [2]:
weather_train_path = "../data/weather_dataset.csv"

In [3]:
weather_train = pd.read_csv(weather_train_path, sep=";", index_col=0)

/var/folders/n5/zftt7r7n09j715hq4n1wqyjmmqxvb8/T/ipykernel_87971/907385527.py:1: DtypeWarning: Columns (22,29) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_train = pd.read_csv(weather_train_path, sep=";", index_col=0)


In [4]:
weather_train.station_id.unique()

array([22550, 26063, 22820, 34009, 34123, 27947, 26898])

In [5]:
data_list = []
target_list = []

for key, data in tqdm(weather_train.groupby("station_id")):
    data_x, data_y = prepare_dataset.prepare_dataset(data)
    data_y["station_id"] = key
    data_y["local_time"] = data_x.local_time

    data_list.append(data_x)
    target_list.append(data_y)

  0%|          | 0/7 [00:00<?, ?it/s]

2022-12-12 00:00:26.794 | INFO     | prepare_dataset:prepare_dataset:96 - Current mode of preparation is processed.


  0%|          | 0/52167 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

2022-12-12 00:00:29.664 | INFO     | prepare_dataset:prepare_dataset:111 - Step: local_time_pipeline
2022-12-12 00:00:29.676 | INFO     | prepare_dataset:prepare_dataset:111 - Step: dd_pipeline


...

2022-12-12 00:01:23.480 | INFO     | prepare_dataset:prepare_dataset:114 - Filling nans!


  0%|          | 0/52167 [00:00<?, ?it/s]

  0%|          | 0/52167 [00:00<?, ?it/s]

  0%|          | 0/52167 [00:00<?, ?it/s]

2022-12-12 00:01:28.457 | INFO     | prepare_dataset:prepare_dataset:117 - Done (processed)!


In [6]:
weather_train_x = pd.concat(data_list, ignore_index=True)
weather_train_y = pd.concat(target_list, ignore_index=True)

In [678]:
weather_train_y["local_time"] = weather_train_x.local_time

In [652]:
weather_train_y

,VETER,SHKVAL,METEL,DOZD,SNEG,GRAD,TUMAN,GOLOLED
0,False,False,False,True,True,False,False,False
1,False,False,False,True,True,False,False,False
2,False,False,False,True,True,False,False,False
3,False,False,False,True,True,False,False,False
4,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...
365164,False,False,False,False,True,False,False,False
365165,False,False,False,False,True,False,False,False
365166,False,False,False,False,True,False,False,False
365167,False,False,False,False,True,False,False,False


In [7]:
def fast_rolling_window(s, window):
    s = s.values
    rolling = view_as_windows(s, window)
    first_row = rolling[0].astype(float)
    temp = np.triu(scipy.linalg.toeplitz(first_row)).T
    temp[np.triu_indices(3, 1)] = np.nan
    temp = np.fliplr(temp)[:-1]

    return np.vstack([temp, rolling])

In [8]:
numeric_features = []
categorial_features = []

truncated = weather_train_x.iloc[:10]


for col in tqdm(truncated.columns):
    if col in ["local_time", "station_id"]:
        continue
    try:
        truncated[col].mean()
        numeric_features.append(col)
    except:
        categorial_features.append(col)

  0%|          | 0/38 [00:00<?, ?it/s]

In [17]:
numeric_features

['T',
 'Po',
 'P',
 'Pa',
 'U',
 'Ff',
 'VV',
 'Td',
 'tR',
 'sss',
 'dd_isnan',
 'dd_changed',
 'dd_x_rad',
 'dd_y_rad',
 'Tn_isnan',
 'Tn_val',
 'Tx_isnan',
 'Tx_val',
 'Tg_isnan',
 'Tg_val',
 'ff3_isnan',
 'ff3_val',
 'ff10_isnan',
 'ff10_val',
 'N_isnan',
 'N_val',
 'Nh_isnan',
 'Nh_val',
 'H_isnan',
 'H_val']

In [19]:
weather_train_x.columns

Index(['local_time', 'T', 'Po', 'P', 'Pa', 'U', 'Ff', 'Cl', 'Cm', 'Ch', 'VV',
       'Td', 'RRR', 'tR', 'E', 'E'', 'sss', 'station_id', 'dd_isnan',
       'dd_changed', 'dd_x_rad', 'dd_y_rad', 'Tn_isnan', 'Tn_val', 'Tx_isnan',
       'Tx_val', 'Tg_isnan', 'Tg_val', 'ff3_isnan', 'ff3_val', 'ff10_isnan',
       'ff10_val', 'N_isnan', 'N_val', 'Nh_isnan', 'Nh_val', 'H_isnan',
       'H_val'],
      dtype='object')

In [46]:
from loguru import logger


import scipy.stats as sps


def mad(data, axis=None):
    return np.mean(np.abs(data - np.mean(data, axis, keepdims=True)), axis)


def count_time_no_event(observation: pd.Series):
    observation = ~observation
    tmp = observation.cumsum() - observation.cumsum().where(~observation).ffill().fillna(0).astype(int)
    return tmp.shift(1, fill_value=0)


def get_window_features(df, features, window=8):
    info_rolling = {}
    for feature in features:
        rolling_window = fast_rolling_window(df[feature], window=window)
        info_rolling.update({
            f"{feature}_mean_window={window}": np.mean(rolling_window, axis=-1),
            f"{feature}_median_window={window}": np.median(rolling_window, axis=-1),
            f"{feature}_std_window={window}": np.std(rolling_window, axis=-1, ddof=1),
            f"{feature}_mad_window={window}": mad(rolling_window, axis=-1),
            f"{feature}_skew_window={window}": sps.skew(rolling_window, axis=-1),
            f"{feature}_kurt_window={window}": sps.kurtosis(rolling_window, axis=-1),
            f"{feature}_min_window={window}": np.min(rolling_window, axis=-1),
            f"{feature}_max_window={window}": np.max(rolling_window, axis=-1),
            f"{feature}_count_window={window}": np.count_nonzero(~np.isnan(rolling_window), axis=-1),
            f"{feature}_has_nan_window={window}": np.any(np.isnan(rolling_window), axis=-1).astype(int)
        })

    return pd.DataFrame(info_rolling, index=df.index).fillna(0)


def get_window_features_with_offset(df, target, features, offset):
    target = target.astype(int)
    index = df.index
    index_year_ago = index.intersection(index - offset)

    target_year_ago = target.loc[index_year_ago]
    target.index += offset
    df_new = df.merge(target_year_ago, how="left", left_index=True, right_index=True, suffixes=(None, "_year_ago"))

    df_year_ago = df.loc[index_year_ago, features]
    df_year_ago.index += offset

    df_new = df.merge(df_year_ago, how="left", left_index=True, right_index=True, suffixes=(None, "_year_ago"))

    return df_new.fillna(0)


def make_features_for_station(df, targets, numeric_features, categorial_columns, windows=None):
    df.sort_values("local_time", inplace=True)
    df.reset_index(drop=True, inplace=True)
    targets.sort_values("local_time", inplace=True)
    targets.reset_index(drop=True, inplace=True)

    windows = windows or [8, 80, 360]
    numeric_features = copy.copy(numeric_features)

    new_df = df[numeric_features + categorial_columns + ["station_id", "local_time"]]

    # представим дату как синус и косинус
    logger.info("Coding date as (sin, cos)")
    new_df["sin_day_of_year"] = new_df.local_time.map(lambda x: np.sin(2 * np.pi * x.timetuple().tm_yday / 365))
    new_df["cos_day_of_year"] = new_df.local_time.map(lambda x: np.cos(2 * np.pi * x.timetuple().tm_yday / 365))
    new_df["sin_time_of_day"] = new_df.local_time.map(lambda x: np.sin(2 * np.pi * x.timetuple().tm_hour / 24))
    new_df["cos_time_of_day"] = new_df.local_time.map(lambda x: np.cos(2 * np.pi * x.timetuple().tm_hour / 24))

    # считаем кол-во дней без дождя, снега и тд
    logger.info("Counting time without events")
    for col in tqdm(targets.columns):
        if col in ["station_id", "local_time"]:
            continue
        new_feature_name = f"no_{col}_time"
        new_df[new_feature_name] = count_time_no_event(targets[col].shift().fillna(False))
        numeric_features.append(new_feature_name)

    new_numeric_df = []
    logger.info("Window features")
    for window in tqdm(windows):
        new_numeric_df.append(
            get_window_features(
                new_df, numeric_features, window
            )
        )
    new_df = pd.concat(new_numeric_df + [new_df], axis=1)

    new_df = get_window_features_with_offset(new_df, targets, numeric_features, 2920)
    
    return new_df.iloc[:-1], targets.iloc[1:]

In [54]:
prepared_data_list = []
prepared_target_list = []


weather_train_x[numeric_features] = weather_train_x[numeric_features].astype(float)


for key, data in tqdm(weather_train_x.groupby("station_id")):
    one_station_y = weather_train_y[weather_train_y.station_id == key] 

    data_x, data_y = make_features_for_station(data, one_station_y, numeric_features, categorial_features, [8, 80, 360])

    prepared_data_list.append(data_x)
    prepared_target_list.append(data_y)

  0%|          | 0/7 [00:00<?, ?it/s]

/var/folders/n5/zftt7r7n09j715hq4n1wqyjmmqxvb8/T/ipykernel_87971/1174737086.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets.sort_values("local_time", inplace=True)
2022-12-12 02:27:14.671 | INFO     | __main__:make_features_for_station:66 - Coding date as (sin, cos)
/var/folders/n5/zftt7r7n09j715hq4n1wqyjmmqxvb8/T/ipykernel_87971/1174737086.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["sin_day_of_year"] = new_df.local_time.map(lambda x: np.sin(2 * np.pi * x.timetuple().tm_yday / 365))
/var/folders/n5/zftt7r7n09j715hq4n1wqyjmmqxvb8/T/ip

  0%|          | 0/10 [00:00<?, ?it/s]

/var/folders/n5/zftt7r7n09j715hq4n1wqyjmmqxvb8/T/ipykernel_87971/1174737086.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[new_feature_name] = count_time_no_event(targets[col].shift().fillna(False))
/var/folders/n5/zftt7r7n09j715hq4n1wqyjmmqxvb8/T/ipykernel_87971/1174737086.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[new_feature_name] = count_time_no_event(targets[col].shift().fillna(False))
/var/folders/n5/zftt7r7n09j715hq4n1wqyjmmqxvb8/T/ipykernel_87971/1174737086.py:78: Setti

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

/var/folders/n5/zftt7r7n09j715hq4n1wqyjmmqxvb8/T/ipykernel_87971/1174737086.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets.sort_values("local_time", inplace=True)
2022-12-12 02:29:58.720 | INFO     | __main__:make_features_for_station:66 - Coding date as (sin, cos)
/var/folders/n5/zftt7r7n09j715hq4n1wqyjmmqxvb8/T/ipykernel_87971/1174737086.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["sin_day_of_year"] = new_df.local_time.map(lambda x: np.sin(2 * np.pi * x.timetuple().tm_yday / 365))
/var/folders/n5/zftt7r7n09j715hq4n1wqyjmmqxvb8/T/ip

  0%|          | 0/10 [00:00<?, ?it/s]

/var/folders/n5/zftt7r7n09j715hq4n1wqyjmmqxvb8/T/ipykernel_87971/1174737086.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[new_feature_name] = count_time_no_event(targets[col].shift().fillna(False))
/var/folders/n5/zftt7r7n09j715hq4n1wqyjmmqxvb8/T/ipykernel_87971/1174737086.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[new_feature_name] = count_time_no_event(targets[col].shift().fillna(False))
/var/folders/n5/zftt7r7n09j715hq4n1wqyjmmqxvb8/T/ipykernel_87971/1174737086.py:78: Setti

  0%|          | 0/3 [00:00<?, ?it/s]

In [55]:
all_train = pd.concat(prepared_data_list, ignore_index=True)
all_target = pd.concat(prepared_target_list, ignore_index=True)

In [56]:
all_train.to_csv("../data/my_ready_dataset.csv", index=False)

In [57]:
all_target.to_csv("../data/my_ready_targets.csv", index=False)